Name: Krishna Sharma

Reg No. : AP22110010128

Sec: R

In [45]:
import pandas as pd
import numpy as np
from scipy.stats import mode

In [46]:
size = 100
grades = ['A'] * int(size * 0.3) + ['B'] * int(size * 0.3) + ['C'] * int(size * 0.2) + ['D'] * int(size * 0.2)
salaries = np.random.randint(2000, 100000, size)
df = pd.DataFrame({'Employee_ID': [f'E{i:03d}' for i in range(1, size + 1)], 'Grade': grades, 'Salary': salaries})
df.to_csv('employee_dataset.csv', index=False)
df

,Employee_ID,Grade,Salary
0,E001,A,81883
1,E002,A,57078
2,E003,A,68069
3,E004,A,71024
4,E005,A,93552
...,...,...,...
95,E096,D,45800
96,E097,D,5399
97,E098,D,78744
98,E099,D,14106


In [47]:
def generate(df, sample_size, method, replace=True):
    if method == 'random':
        sample = df.sample(n=sample_size, replace=replace)
    elif method == 'proportional':
        sample = df.groupby('Grade').apply(lambda x: x.sample(n=int(sample_size * (len(x) / len(df))), replace=replace))
    return sample['Salary'].mean(), sample['Salary'].median(), sample['Salary'].mode()[0]

In [48]:
sample_sizes = [30, 40]
methods = ['random', 'proportional']
replacements = [True, False]

results = []
for sample_size in sample_sizes:
    for method in methods:
        for replace in replacements:
            if method == 'proportional' and replace == False and sample_size > df.groupby('Grade').size().min():
                continue
            for _ in range(2 if sample_size == 30 else 3):
                mean, median, mode = generate(df, sample_size, method, replace)
                results.append({'Sample Size': sample_size, 'Method': method, 'Replace': replace, 'Mean': mean, 'Median': median, 'Mode': mode})

C:\Users\krish\AppData\Local\Temp\ipykernel_23844\2191224694.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample = df.groupby('Grade').apply(lambda x: x.sample(n=int(sample_size * (len(x) / len(df))), replace=replace))
C:\Users\krish\AppData\Local\Temp\ipykernel_23844\2191224694.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample = df.groupby('Grade').apply(lambda x: x.sample(n=int(sample_size

In [49]:
results_df = pd.DataFrame(results)
results_df.to_csv('sampling_results.csv', index=False)
results_df

,Sample Size,Method,Replace,Mean,Median,Mode
0,30,random,True,51486.966667,49268.0,49268
1,30,random,True,55979.833333,60716.0,55411
2,30,random,False,52000.200000,50929.5,12191
3,30,random,False,56230.700000,55603.0,15850
4,30,proportional,True,55142.666667,56265.5,33617
5,30,proportional,True,51941.666667,55482.0,15850
6,40,random,True,47472.350000,47969.0,14580
7,40,random,True,49558.975000,54788.0,61509
8,40,random,True,47223.900000,46790.0,6263
9,40,random,False,63319.525000,68066.5,11096


In [50]:
random_samples = {
    "size_30_1": df.sample(n=30, replace=True),
    "size_30_2": df.sample(n=30, replace=True),
    "size_40_1": df.sample(n=40, replace=True),
    "size_40_2": df.sample(n=40, replace=True),
    "size_40_3": df.sample(n=40, replace=True)
}

In [51]:
def proportional_sample(data, size):    
    proportions = {
        'A': 0.3,
        'B': 0.3,
        'C': 0.2,
        'D': 0.2
    }
    samples = []
    for grade, prop in proportions.items():
        n = int(size * prop)
        grade_sample = data[data['Grade'] == grade].sample(n=n, replace=True)
        samples.append(grade_sample)
    return pd.concat(samples)

proportional_samples_replace = {
    "size_30_1": proportional_sample(df, 30),
    "size_30_2": proportional_sample(df, 30),
    "size_40_1": proportional_sample(df, 40),
    "size_40_2": proportional_sample(df, 40),
    "size_40_3": proportional_sample(df, 40)
}

In [52]:
def proportional_sample_without_replacement(data, size):
    proportions = {
        'A': 0.3,
        'B': 0.3,
        'C': 0.2,
        'D': 0.2
    }
    samples = []
    for grade, prop in proportions.items():
        n = int(size * prop)
        grade_sample = data[data['Grade'] == grade].sample(n=n, replace=False)
        samples.append(grade_sample)
    return pd.concat(samples)

proportional_samples_no_replace = {
    "size_30": proportional_sample_without_replacement(df, 30),
    "size_40": proportional_sample_without_replacement(df, 40)
}

In [53]:
def calculate_statistics(sample):
    mean_salary = sample['Salary'].mean()
    median_salary = sample['Salary'].median()
    mode_salary = sample['Salary'].mode().iloc[0]  
    return mean_salary, median_salary, mode_salary

In [54]:
results = {}

# Random samples
for key, sample in random_samples.items():
    results[key] = calculate_statistics(sample)

# Proportional allocation with replacement
for key, sample in proportional_samples_replace.items():
    results[key] = calculate_statistics(sample)

# Proportional allocation without replacement
for key, sample in proportional_samples_no_replace.items():
    results[key] = calculate_statistics(sample)

In [55]:
results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Mean', 'Median', 'Mode'])
results_df

,Mean,Median,Mode
size_30_1,60975.233333,63602.0,61730
size_30_2,51188.900000,48147.0,19785
size_40_1,56189.425000,58424.5,18037
size_40_2,60443.050000,56436.5,94012
size_40_3,50614.125000,54549.0,14580
size_30,50420.633333,54935.0,6560
size_40,53548.200000,54549.0,5399


In [56]:
import pandas as pd
import numpy as np

In [57]:
size = 100
grades = ['A'] * int(size * 0.3) + ['B'] * int(size * 0.3) + ['C'] * int(size * 0.2) + ['D'] * int(size * 0.2)
salaries = np.random.randint(2000, 100000, size)
df = pd.DataFrame({'Employee_ID': [f'E{i:03d}' for i in range(1, size + 1)], 
                   'Grade': grades, 'Salary': salaries})
df.to_csv('employee_dataset.csv', index=False)

In [58]:
def generate_sample(df, sample_size, method, replace=True, proportions=None):
    if method == 'random':
        return df.sample(n=sample_size, replace=replace)
    elif method == 'proportional':
        if not proportions:
            raise ValueError("Proportions are required for proportional sampling.")
        samples = []
        for grade, prop in proportions.items():
            n = int(sample_size * prop)
            if replace or n <= len(df[df['Grade'] == grade]):
                samples.append(df[df['Grade'] == grade].sample(n=n, replace=replace))
        return pd.concat(samples)

In [59]:
def calculate_statistics(sample):
    mean = sample['Salary'].mean()
    median = sample['Salary'].median()
    mode = sample['Salary'].mode().iloc[0]
    return mean, median, mode

In [60]:
proportions = {'A': 0.3, 'B': 0.3, 'C': 0.2, 'D': 0.2}
results = []

In [61]:
for size in [30, 40]:
    for method in ['random', 'proportional']:
        for replace in [True, False]:
            if method == 'proportional' and not replace and size > df['Grade'].value_counts().min():
                continue
            for _ in range(2 if size == 30 else 3):
                sample = generate_sample(df, size, method, replace, proportions)
                mean, median, mode = calculate_statistics(sample)
                results.append({'Sample Size': size, 'Method': method, 'Replace': replace, 
                                'Mean': mean, 'Median': median, 'Mode': mode})

In [62]:
results_df = pd.DataFrame(results)
results_df.to_csv('sampling_results.csv', index=False)
results_df

,Sample Size,Method,Replace,Mean,Median,Mode
0,30,random,True,51360.033333,57815.0,59855
1,30,random,True,54585.466667,59790.0,97654
2,30,random,False,51701.633333,45597.0,4957
3,30,random,False,67945.666667,76489.5,5478
4,30,proportional,True,54938.233333,61583.0,17587
5,30,proportional,True,63954.233333,76611.5,86326
6,40,random,True,58664.825000,65654.5,48926
7,40,random,True,57842.375000,61653.0,28417
8,40,random,True,52295.350000,53882.0,3574
9,40,random,False,62399.600000,70297.5,4957
